# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor

**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [2]:
df_ski = pd.read_csv('step3_output.csv', index_col = 0)

In [3]:
df_ski.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,Clusters
0,Alyeska Resort,Alaska,3939,2500,1,0.0,0,2,2,0,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,1
1,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,1
2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
3,Arizona Snowbowl,Arizona,11500,2300,0,0.0,1,0,2,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,0
4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,0


In [4]:
df_ski.shape

(329, 26)

In [5]:
df_ski.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 329
Data columns (total 26 columns):
Name                 329 non-null object
state                329 non-null object
summit_elev          329 non-null int64
vertical_drop        329 non-null int64
trams                329 non-null int64
fastEight            329 non-null float64
fastSixes            329 non-null int64
fastQuads            329 non-null int64
quad                 329 non-null int64
triple               329 non-null int64
double               329 non-null int64
surface              329 non-null int64
total_chairs         329 non-null int64
Runs                 329 non-null float64
TerrainParks         329 non-null float64
LongestRun_mi        329 non-null float64
SkiableTerrain_ac    329 non-null float64
Snow Making_ac       329 non-null float64
daysOpenLastYear     329 non-null float64
yearsOpen            329 non-null float64
averageSnowfall      329 non-null float64
AdultWeekday         329 non-null f

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [6]:
#pd.concat([df_ski.drop('state', axis=1), pd.get_dummies(df_ski['state'])], axis=1)
df_ski_dum = pd.get_dummies(df_ski, columns=['state'])

In [7]:
df_ski_dum.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,Alyeska Resort,3939,2500,1,0.0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,0,0.0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,0,0.0,1,0,2,2,1,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort, which you don't need in the dataframe for modeling so it can be droppped here as well. You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [8]:
df = df_ski_dum.drop('Name', axis=1)

In [9]:
#Separeting the response variables from the features
res_var = ['AdultWeekend', 'AdultWeekday', 'daysOpenLastYear','projectedDaysOpen']
feat_var = list(df.columns)
feat_var.pop(feat_var.index('AdultWeekend'))

'AdultWeekend'

In [10]:
'AdultWeekend' in feat_var

False

In [11]:
df_res = df[res_var]
df_feat = df[feat_var]

In [12]:
'AdultWeekedn' in df_feat.columns

False

In [13]:
y = df_res['AdultWeekend']

In [14]:
x = df_feat

In [15]:
#x_scaled = preprocessing.scale(df_feat.values)
sc = preprocessing.StandardScaler()
X_scaled = sc.fit_transform(x)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selectionm import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [16]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.25, random_state=0)

In [17]:
X_test[:20]

array([[-0.86948109, -1.04513212, -0.30945373, ..., -0.11094004,
        -0.22609337, -0.15786741],
       [-1.01935525, -1.00586293, -0.30945373, ..., -0.11094004,
        -0.22609337, -0.15786741],
       [ 0.96816518,  1.24203294, -0.30945373, ..., -0.11094004,
        -0.22609337, -0.15786741],
       ...,
       [-0.87353174, -1.03982547, -0.30945373, ..., -0.11094004,
        -0.22609337, -0.15786741],
       [ 0.46858463,  1.05099363,  1.4766915 , ..., -0.11094004,
        -0.22609337, -0.15786741],
       [-0.85057804, -0.74795986, -0.30945373, ..., -0.11094004,
        -0.22609337, -0.15786741]])

Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [18]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [19]:
#We can use lm in order to get y_pred
y_pred = lm.predict(X_test)
#score
print(lm.score(X_test, y_test))
print(lm.score(X_test, y_pred))
#We then have to measure the difference between y_pred and y_test
print(explained_variance_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

-8.72936720592153e+20
1.0
-8.624194107059786e+20
71089825003.15166


In [20]:
y_pred

array([6.20517960e+01, 2.94404678e+01, 4.96794815e+01, 4.12475967e+01,
       4.71206436e+01, 6.48120499e+01, 1.08564980e+02, 5.00996475e+01,
       5.84477921e+01, 4.43667374e+01, 1.19882362e+02, 7.29092178e+01,
       8.77046280e+01, 4.14377823e+01, 8.74585342e+01, 4.16528702e+01,
       9.16526260e+01, 5.86733780e+01, 5.43227921e+01, 5.02449112e+01,
       5.61743546e+01, 5.54018936e+01, 8.79087296e+01, 6.46391983e+01,
       4.47399063e+01, 7.33066788e+01, 3.46191788e+01, 6.29390030e+01,
       7.39409561e+01, 5.45185928e+01, 5.96951065e+01, 5.43630753e+01,
       6.34194717e+01, 5.81055069e+01, 6.11177139e+01, 3.19975967e+01,
       4.38765030e+01, 7.38208389e+01, 5.37148819e+01, 4.70893936e+01,
       5.27788467e+01, 6.27964249e+01, 6.24502335e+01, 1.62138222e+02,
       6.06583633e+01, 5.07104874e+01, 9.09431534e+01, 8.73818741e+01,
       3.65474014e+01, 6.38149796e+01, 6.59917374e+01, 3.99897842e+01,
       4.16753311e+01, 8.09253311e+01, 6.86699600e+01, 7.88574600e+01,
      

In [21]:
y_test

123    64.16681
67     35.00000
143    49.00000
247    33.00000
147    50.00000
         ...   
34     25.00000
134    48.00000
133    46.00000
98     48.00000
259    65.00000
Name: AdultWeekend, Length: 83, dtype: float64

## Review Model Outcomes — Iterate over additional models as needed.

In [22]:
#Trying a Random Forest Regression on the data
f_regress= RandomForestRegressor(random_state = 0, n_estimators=200)
f_regress.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [23]:
print(f_regress.score(X_test, y_test))
fy_pred = f_regress.predict(X_test)
print(explained_variance_score(y_test, fy_pred))
print(mean_absolute_error(y_test, fy_pred))

0.8571013363879354
0.8574590782455961
6.247582128514064


In [24]:
#The linear model seems to be thrown off because of the states. Looks like a Forest Regression would be able
#to handle taking the states into account better

**<font color='teal'> Print the intercept value from the linear model. </font>**

In [25]:
model.intercept_

17934515184.91996

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [26]:
lm.coef_

array([ 1.98914243e+00,  2.09983047e-01, -9.61433808e+11, -9.45316377e+10,
       -1.11897612e+12, -3.77380802e+12, -2.25172723e+12, -2.77682700e+12,
       -3.11587380e+12, -3.52818858e+12,  9.93398170e+12,  1.61682129e-01,
        2.18750000e-01, -1.45912170e-01, -1.29223633e+00, -6.32324219e-02,
        6.20361328e-01,  3.76953125e-01,  5.20019531e-01,  1.93927612e+01,
       -4.77783203e-01,  1.11633301e+00, -1.04125977e+00, -5.60865940e+11,
       -4.58646954e+11, -1.44236324e+12, -1.44236324e+12, -7.21850309e+11,
       -1.10613948e+12, -6.46638139e+11, -4.58646954e+11, -5.60865940e+11,
       -9.62467079e+11, -3.24807883e+11, -1.06071703e+12, -1.67282021e+12,
       -1.19099225e+12, -4.58646954e+11, -1.10613948e+12, -6.46638139e+11,
       -1.26917585e+12, -4.58646954e+11, -9.62467079e+11, -1.77252539e+12,
       -7.89526163e+11, -7.21850309e+11, -1.01294294e+12, -1.37640831e+12,
       -3.24807883e+11, -4.58646954e+11,  1.87483350e-01, -1.14948910e+12,
       -1.23083572e+12, -

In [27]:
x_cols = x.columns
x.columns

Index(['summit_elev', 'vertical_drop', 'trams', 'fastEight', 'fastSixes',
       'fastQuads', 'quad', 'triple', 'double', 'surface', 'total_chairs',
       'Runs', 'TerrainParks', 'LongestRun_mi', 'SkiableTerrain_ac',
       'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen', 'averageSnowfall',
       'AdultWeekday', 'projectedDaysOpen', 'NightSkiing_ac', 'Clusters',
       'state_Alaska', 'state_Arizona', 'state_California', 'state_Colorado',
       'state_Connecticut', 'state_Idaho', 'state_Illinois', 'state_Indiana',
       'state_Iowa', 'state_Maine', 'state_Maryland', 'state_Massachusetts',
       'state_Michigan', 'state_Minnesota', 'state_Missouri', 'state_Montana',
       'state_Nevada', 'state_New Hampshire', 'state_New Jersey',
       'state_New Mexico', 'state_New York', 'state_North Carolina',
       'state_Ohio', 'state_Oregon', 'state_Pennsylvania',
       'state_Rhode Island', 'state_South Dakota', 'state_Tennessee',
       'state_Utah', 'state_Vermont', 'state_Virginia',

In [28]:
len(x_cols)

58

In [29]:
len(lm.coef_)

58

In [30]:
co_dict = {'feature':[], 'coef':[]}
for feat, coef in zip(x_cols, lm.coef_):
    co_dict['feature'].append(feat)
    co_dict['coef'].append(abs(coef))
    

In [31]:
df_try = pd.DataFrame(co_dict)


In [32]:
df_try.sort_values(by=['coef'], ascending=False).head(10)

,feature,coef
10,total_chairs,9.933982e+12
5,fastQuads,3.773808e+12
9,surface,3.528189e+12
8,double,3.115874e+12
7,triple,2.776827e+12
6,quad,2.251727e+12
43,state_New York,1.772525e+12
35,state_Michigan,1.672820e+12
25,state_California,1.442363e+12
26,state_Colorado,1.442363e+12


In [33]:
def feature_coef(lst, arry):
    co_dict = {'feature':[], 'coef':[]}
    for feat, coef in zip(lst, arry):
        co_dict['feature'].append(feat)
        co_dict['coef'].append(abs(coef))
    df_tst = pd.DataFrame(co_dict)
    return df_tst.sort_values(by=['coef'], ascending=False).head(10)

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [34]:
df2 = df_ski.drop(columns=['state', 'Name'], axis=1)

In [35]:
df2.head()

,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,Clusters
0,3939,2500,1,0.0,0,2,2,0,0,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,1
1,2600,1540,0,0.0,0,0,0,0,4,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,1
2,2090,294,0,0.0,0,0,0,1,0,2,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
3,11500,2300,0,0.0,1,0,2,2,1,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,0
4,11100,1800,0,0.0,0,1,2,3,1,0,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,0


In [36]:
feat_var2 = list(df2.columns)
feat_var2.pop(feat_var2.index('AdultWeekend'))

'AdultWeekend'

In [37]:
df2_res = df2[res_var]
df2_feat = df2[feat_var2]

In [38]:
y2 = df2_res['AdultWeekend']
x2 = df2_feat

In [39]:
x2_scaled = sc.fit_transform(x2)

In [40]:
X2_train, X2_test, y2_train, y2_test = model_selection.train_test_split(x2_scaled, y2, test_size=0.25, random_state=0)

In [41]:
lm = linear_model.LinearRegression()
model2 = lm.fit(X2_train,y2_train)

In [42]:
#We can use lm in order to get y_pred
y2_pred = model2.predict(X2_test)
#scores
print(model2.score(X2_test, y2_test))
print(model2.score(X2_test, y2_pred))
#We then have to measure the difference between y_pred and y_test
print(explained_variance_score(y2_test, y2_pred))
print(mean_absolute_error(y2_test, y2_pred))

0.9033687686770085
1.0
0.9058481162578224
4.992542206242077


In [43]:
y2_pred

array([ 63.89664881,  36.29804492,  54.73292629,  39.24092654,
        52.63171844,  71.80084404, 108.59354987,  54.12776939,
        63.26097604,  46.11578814, 119.74911093,  80.25133219,
        85.57720812,  50.6155392 ,  89.08544679,  40.34471775,
        88.93419015,  61.19068728,  59.26082009,  51.27455543,
        58.14040006,  58.41981788,  84.72721977,  62.3944257 ,
        52.94370417,  71.42994303,  35.1084049 ,  49.40203128,
        76.37423535,  60.83550033,  59.49589311,  51.91105627,
        48.03112897,  60.58893318,  59.49221285,  33.92782099,
        47.50788721,  73.05668791,  57.90085598,  53.73521861,
        51.52634958,  62.71048329,  67.4884604 , 164.5609695 ,
        61.7025061 ,  59.52730288,  85.96707129,  83.58615916,
        38.56870177,  67.06140732,  67.79594815,  45.20191789,
        38.19306308,  78.59093992,  71.45600302,  78.66853589,
        45.47633581,  60.26902259, 114.6685108 ,  62.72515638,
        71.71505603,  60.01559521,  43.88316324,  64.13

In [44]:
#intercept of the second model
model2.intercept_

63.8463610811061

In [45]:
#Exploring the coefficients that are weighing heavily in our model
feature_coef(x2.columns, model2.coef_)

,feature,coef
19,AdultWeekday,19.759043
0,summit_elev,2.434033
1,vertical_drop,2.069286
14,SkiableTerrain_ac,1.952410
7,triple,1.548715
6,quad,1.217232
22,Clusters,1.205379
21,NightSkiing_ac,1.106598
16,daysOpenLastYear,1.103192
9,surface,0.910436


**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [46]:
df3 = df_ski.drop(columns=['state', 'Name', 'summit_elev'], axis=1)
df3.head()

,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,total_chairs,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,Clusters
0,2500,1,0.0,0,2,2,0,0,2,7,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,1
1,1540,0,0.0,0,0,0,0,4,0,4,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,1
2,294,0,0.0,0,0,0,1,0,2,3,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
3,2300,0,0.0,1,0,2,2,1,2,8,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,0
4,1800,0,0.0,0,1,2,3,1,0,7,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,0


In [47]:
feat_var3 = list(df3.columns)
feat_var3.pop(feat_var3.index('AdultWeekend'))

'AdultWeekend'

In [48]:
df3_res = df3[res_var]
df3_feat = df3[feat_var3]

In [49]:
y3 = df3_res['AdultWeekend']
x3 = df3_feat
x3_scaled = sc.fit_transform(x3)

In [50]:
X3_train, X3_test, y3_train, y3_test = model_selection.train_test_split(x3_scaled, y3, test_size=0.25, random_state=0)

In [51]:
lm = linear_model.LinearRegression()
model3 = lm.fit(X3_train,y3_train)

In [52]:
#We can use lm in order to get y_pred
y3_pred = model3.predict(X3_test)
#scores
print(model3.score(X3_test, y3_test))
print(model3.score(X3_test, y3_pred))
#We then have to measure the difference between y_pred and y_test
print(explained_variance_score(y3_test, y3_pred))
print(mean_absolute_error(y3_test, y3_pred))

0.908404882281127
1.0
0.9099055055630563
5.005431097922615


In [53]:
#The intercept for the model
model3.intercept_

63.907497875712316

In [54]:
#Exploring the coefficients that are weighing heavily in our model
feature_coef(x3.columns, model3.coef_)

,feature,coef
18,AdultWeekday,19.684040
13,SkiableTerrain_ac,2.170864
5,quad,1.476765
20,NightSkiing_ac,1.445014
6,triple,1.366631
21,Clusters,1.039030
15,daysOpenLastYear,1.019666
8,surface,0.953346
4,fastQuads,0.898182
17,averageSnowfall,0.882908


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | NA | NA |-|
| Model 2. | 0.9058|4.9925 |'state'|
| Model 3. | 0.9099 |5.0054 |'state','summit_elev','base_elev'|

Model Selection:

Model 3 seems to be the best model. It edges model 2 when it comes to explained variance. Also model 3 discards features that are outside of management's control. The top ten features with the highest coefficients include AdultWeekday prices and daysOpenLastYear. 